In [35]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from skimage import io

In [36]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import  transforms, datasets

In [37]:
class Labeling(nn.Module):
    def __init__(self):
        super(Labeling, self).__init__()
        
        def max_pooling_3d():
            return nn.MaxPool3d(kernel_size=2, stride=2, padding=0)
        self.init = max_pooling_3d()
    
    def forward(self, x):
       # Down sampling
        out = self.init(x)
        return out

In [38]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        def conv_block_3d(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm3d(out_channels),
                nn.LeakyReLU(0.2, inplace=True),)


        def conv_trans_block_3d(in_channels, out_channels):
            return nn.Sequential(
                nn.ConvTranspose3d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.BatchNorm3d(out_channels),
                nn.LeakyReLU(0.2, inplace=True),)


        def max_pooling_3d():
            return nn.MaxPool3d(kernel_size=2, stride=2, padding=0)


        def conv_block_2_3d(in_channels, out_channels):
            return nn.Sequential(
                conv_block_3d(in_channels, out_channels),
                nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm3d(out_channels),)

#         def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
#             layers = []
#             layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
#                                  kernel_size=kernel_size, stride=stride, padding=padding,
#                                  bias=bias)]
#             layers += [nn.BatchNorm2d(num_features=out_channels)]
#             layers += [nn.ReLU()]

#             cbr = nn.Sequential(*layers) # *으로 list unpacking 

#             return cbr
        
    ######Define layers
        
        # Down sampling
        self.init = max_pooling_3d()
        self.down_1 = conv_block_2_3d(in_channels=1, out_channels=4)
        self.pool_1 = max_pooling_3d()
        self.down_2 = conv_block_2_3d(in_channels=4, out_channels=8)
        self.pool_2 = max_pooling_3d()
        self.down_3 = conv_block_2_3d(in_channels=8, out_channels=16)
        self.pool_3 = max_pooling_3d()
        self.down_4 = conv_block_2_3d(in_channels=16, out_channels=32)
        self.pool_4 = max_pooling_3d()
        self.down_5 = conv_block_2_3d(in_channels=32, out_channels=64)
        self.pool_5 = max_pooling_3d()
        
        
        # Bridge
        self.bridge = conv_block_2_3d(in_channels=64, out_channels=128)

        
        # Up sampling
        self.trans_1 = conv_trans_block_3d(in_channels=128, out_channels=128)
        self.up_1 = conv_block_2_3d(in_channels=192, out_channels=64)
        self.trans_2 = conv_trans_block_3d(in_channels=64, out_channels=64)
        self.up_2 = conv_block_2_3d(in_channels=96, out_channels=32)
        self.trans_3 = conv_trans_block_3d(in_channels=32, out_channels=32)
        self.up_3 = conv_block_2_3d(in_channels=48, out_channels=16)
        self.trans_4 = conv_trans_block_3d(in_channels=16, out_channels=16)
        self.up_4 = conv_block_2_3d(in_channels=24, out_channels=8)
        self.trans_5 = conv_trans_block_3d(in_channels=8, out_channels=8)
        self.up_5 = conv_block_2_3d(in_channels=12, out_channels=4)
        
        
        # Output
        self.out = conv_block_3d(in_channels=4, out_channels=1)
        
    #forwarding
    def forward(self, x):
       # Down sampling
        init = self.init(x)
        down_1 = self.down_1(init) # -> [1, 4, 128, 128, 128]
        pool_1 = self.pool_1(down_1) # -> [1, 4, 64, 64, 64]

        down_2 = self.down_2(pool_1) # -> [1, 8, 64, 64, 64]
        pool_2 = self.pool_2(down_2) # -> [1, 8, 32, 32, 32]

        down_3 = self.down_3(pool_2) # -> [1, 16, 32, 32, 32]
        pool_3 = self.pool_3(down_3) # -> [1, 16, 16, 16, 16]

        down_4 = self.down_4(pool_3) # -> [1, 32, 16, 16, 16]
        pool_4 = self.pool_4(down_4) # -> [1, 32, 8, 8, 8]

        down_5 = self.down_5(pool_4) # -> [1, 64, 8, 8, 8]
        pool_5 = self.pool_5(down_5) # -> [1, 64, 4, 4, 4]

        # Bridge
        bridge = self.bridge(pool_5) # -> [1, 128, 4, 4, 4]
       
        # Up sampling
        trans_1 = self.trans_1(bridge) # -> [1, 128, 8, 8, 8]
        concat_1 = torch.cat([trans_1, down_5], dim=1) # -> [1, 192, 8, 8, 8]
        up_1 = self.up_1(concat_1) # -> [1, 64, 8, 8, 8]

        trans_2 = self.trans_2(up_1) # -> [1, 64, 16, 16, 16]
        concat_2 = torch.cat([trans_2, down_4], dim=1) # -> [1, 96, 16, 16, 16]
        up_2 = self.up_2(concat_2) # -> [1, 32, 16, 16, 16]
   
        trans_3 = self.trans_3(up_2) # -> [1, 32, 32, 32, 32]
        concat_3 = torch.cat([trans_3, down_3], dim=1) # -> [1, 48, 32, 32, 32]
        up_3 = self.up_3(concat_3) # -> [1, 16, 32, 32, 32]
      
        trans_4 = self.trans_4(up_3) # -> [1, 16, 64, 64, 64]
        concat_4 = torch.cat([trans_4, down_2], dim=1) # -> [1, 24, 64, 64, 64]
        up_4 = self.up_4(concat_4) # -> [1, 8, 64, 64, 64]

        trans_5 = self.trans_5(up_4) # -> [1, 8, 128, 128, 128]
        concat_5 = torch.cat([trans_5, down_1], dim=1) # -> [1, 12, 128, 128, 128]
        up_5 = self.up_5(concat_5) # -> [1, 4, 128, 128, 128]
      
        # Output
        out = self.out(up_5) # -> [1, 3, 128, 128, 128]
        return out

        

In [33]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        
        lst_data = os.listdir(self.data_dir)
        
        # 문자열 검사해서 'label'이 있으면 True 
        # 문자열 검사해서 'train'이 있으면 True
        lst_label = [f for f in lst_data if f.startswith('label')] 
        lst_input = [f for f in lst_data if f.startswith('train')] 
        
        lst_label.sort()
        lst_input.sort()
        self.lst_label = lst_label
        self.lst_input = lst_input
    
    def __len__(self):
        return len(self.lst_label)
    
    # 데이터 load 파트
    def __getitem__(self, index):

        Dim_size=np.array((1024,1024,62),dtype=np.int)
        
        f = io.imread(os.path.join(self.data_dir, self.lst_label[index]))
        label=np.array(f)
        
        g = io.imread(os.path.join(self.data_dir, self.lst_input[index]))
        inputs=np.array(g)
    
        
        # normalize, 이미지는 0~255 값을 가지고 있어 이를 0~1사이로 scaling
        #label = label/255.0, label은 어차피 1, input은 최대값을 기준으로 normalize
        inputs = inputs/np.max(inputs)
        label = label.astype(np.float32)
        inputs = inputs.astype(np.float32) 

        
        # 인풋 데이터 차원이 2이면, 채널 축을 추가해줘야한다. 
        # 파이토치 인풋 format (batch, 채널, z, 행, 열)
        
        if label.ndim == 3:
            label = label[:,:,:,np.newaxis]    #파이토치 인풋 포맷을 보고도 맨 뒤에 새로운 축을 생성하는 이유는 다음 class에서 확인하기
        if inputs.ndim == 3:  
            inputs = inputs[:,:,:,np.newaxis] 
        
        data = {'input':inputs, 'label':label}

        # transform에 할당된 class 들이 호출되면서 __call__ 함수 실행
        if self.transform:
            data = self.transform(data)
    
        return data

In [6]:
class ToTensor(object):
    def __call__(self, data):
        label, inputs = data['label'], data['input']
   
        # numpy와 tensor의 배열 차원 순서가 다르다. 
        # numpy : (행, 열, 채널)
        # tensor : (채널, 행, 열)
        # 따라서 위 순서에 맞춰 transpose
        
        label = label.transpose((3, 0, 1, 2)).astype(np.float32) 
        inputs = inputs.transpose((3, 0, 1, 2)).astype(np.float32)
        
        # 이후 np를 tensor로 바꾸는 코드는 다음과 같이 간단하다.
        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(inputs)}

        return data

In [7]:
## 하이퍼 파라미터 설정
lr = 1e-3
batch_size = 1
num_epoch = 50

data_dir = './data'
ckpt_dir = './ckpt12313123'
log_dir = './log'
res_dir = './result'

patience = 1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# transform 적용해서 데이터 셋 불러오기
#transform = transforms.Compose(transforms.Normalize(0.5, 0.5), ToTensor())
dataset_train = Dataset(data_dir=os.path.join(data_dir,'train'),transform=ToTensor())

# 불러온 데이터셋, 배치 size줘서 DataLoader 해주기
loader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle=True)
# #for i, data in enumerate(loader_train,1):
# #    print(data['label'].size())
# # val set도 동일하게 진행
# dataset_val = Dataset(data_dir=os.path.join(data_dir,'val'),transform = ToTensor())
# loader_val = DataLoader(dataset_val, batch_size=1 , shuffle=True)

# 네트워크 불러오기
net = UNet().to(device) # device : cpu or gpu
lala = Labeling().to(device)
# loss 정의
fn_loss = nn.MSELoss().to(device)

# Optimizer 정의
optim = torch.optim.Adam(net.parameters(), lr = lr ) 

# 기타 variables 설정
num_train = len(dataset_train)
#num_val = len(dataset_val)

num_train_for_epoch = np.ceil(num_train/batch_size) # np.ceil : 소수점 반올림
#num_val_for_epoch = np.ceil(num_val/batch_size)

# 기타 function 설정
fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0,2,3,4,1) # device 위에 올라간 텐서를 detach 한 뒤 numpy로 변환
fn_denorm = lambda x, mean, std : (x * std) + mean 
fn_classifier = lambda x :  1.0 * (x > 0.5)  # threshold 0.5 기준으로 indicator function으로 classifier 구현

# Tensorbord
#writer_train = SummaryWriter(log_dir = os.path.join(log_dir,'train'))
#writer_val = SummaryWriter(log_dir = os.path.join(log_dir,'val'))

In [9]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save({'net':net.state_dict()},'%s'%(self.path))
        self.val_loss_min = val_loss

In [10]:
# 네트워크 저장하기
# train을 마친 네트워크 저장 
# net : 네트워크 파라미터, optim  두개를 dict 형태로 저장
def save(ckpt_dir,net,optim,epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net':net.state_dict(),'optim':optim.state_dict()},'%s/model_epoch%d.pth'%(ckpt_dir,epoch))

# 네트워크 불러오기
def load(ckpt_dir,net,optim):
    if not os.path.exists(ckpt_dir): # 저장된 네트워크가 없다면 인풋을 그대로 반환
        epoch = 0
        return net, optim, epoch
    
    ckpt_lst = os.listdir(ckpt_dir) # ckpt_dir 아래 있는 모든 파일 리스트를 받아온다
    ckpt_lst.sort(key = lambda f : int(''.join(filter(str,isdigit,f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir,ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net,optim,epoch

#early_stopping = EarlyStopping(patience = patience, verbose = True)

# 네트워크 학습시키기
start_epoch = 0
net, optim, start_epoch = load(ckpt_dir = ckpt_dir, net = net, optim = optim) # 저장된 네트워크 불러오기


for epoch in range(start_epoch+1,num_epoch +1):
    net.train()
    #loss_arr = []

    for batch, data in enumerate(loader_train,1): # 1은 뭐니 > index start point
        # forward
        label = data['label'].to(device)   # 데이터 device로 올리기  
        label_fin = lala(label)
        inputs = data['input'].to(device)
        output = net(inputs) 
        #plt.imshow(index[0,0,:,:], cmap='gray')

        # backward
        optim.zero_grad()  # gradient 초기화
        loss = fn_loss(output, label_fin)  # output과 label 사이의 loss 계산
        loss.backward() # gradient backpropagation
        optim.step() # backpropa 된 gradient를 이용해서 각 layer의 parameters update

        # save loss
        print('epoch: {}, loss: {}'.format(epoch,loss.item()))
        #loss_arr += [loss.item()]

        # tensorbord에 결과값들 저정하기
        label = fn_tonumpy(label)
        inputs = fn_tonumpy(fn_denorm(inputs,0.5,0.5))
        output = fn_tonumpy(fn_classifier(output))

#         writer_train.add_image('label', label, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
#         writer_train.add_image('input', inputs, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
#         writer_train.add_image('output', output, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

#     writer_train.add_scalar('loss', np.mean(loss_arr), epoch)

    
#     # validation
#     with torch.no_grad(): # validation 이기 때문에 backpropa 진행 x, 학습된 네트워크가 정답과 얼마나 가까운지 loss만 계산
#         net.eval() # 네트워크를 evaluation 용으로 선언
#         loss_arr = []

#         for batch, data in enumerate(loader_val,1):
#             # forward
#             label = data['label'].to(device)
#             inputs = data['input'].to(device)
#             output = net(inputs)

#             # loss 
#             loss = fn_loss(fn_classifier(output),label)
#             loss_arr += [loss.item()]     
#             print('valid : epoch %04d / %04d | Batch %04d \ %f | Loss %f'%(epoch,num_epoch,batch,num_val_for_epoch,np.mean(loss_arr)))

#             # Tensorboard 저장하기
#             label = fn_tonumpy(label)
#             inputs = fn_tonumpy(fn_denorm(inputs, mean=0.5, std=0.5))
#             output = fn_tonumpy(fn_classifier(output))

#             writer_val.add_image('label', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
#             writer_val.add_image('input', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
#             writer_val.add_image('output', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

#         writer_val.add_scalar('loss', np.mean(loss_arr), epoch)
        
#         early_stopping(np.mean(loss_arr), net)

#         if early_stopping.early_stop:
#             print("Early stopping")
#             break
            
        # epoch이 끝날때 마다 네트워크 저장
    save(ckpt_dir=ckpt_dir, net = net, optim = optim, epoch = epoch)

    torch.cuda.empty_cache()
# writer_train.close()
# writer_val.close()

epoch: 1, loss: 1641.90380859375
epoch: 2, loss: 1621.94482421875
epoch: 3, loss: 1598.786865234375
epoch: 4, loss: 1587.117919921875
epoch: 5, loss: 1579.581787109375
epoch: 6, loss: 1575.038330078125
epoch: 7, loss: 1572.3453369140625
epoch: 8, loss: 1570.3753662109375
epoch: 9, loss: 1569.4893798828125
epoch: 10, loss: 1568.77001953125
epoch: 11, loss: 1568.35888671875
epoch: 12, loss: 1568.012451171875
epoch: 13, loss: 1567.57080078125
epoch: 14, loss: 1566.97314453125
epoch: 15, loss: 1566.761962890625
epoch: 16, loss: 1566.5440673828125
epoch: 17, loss: 1566.208251953125
epoch: 18, loss: 1565.9296875
epoch: 19, loss: 1565.6724853515625
epoch: 20, loss: 1565.43359375
epoch: 21, loss: 1565.208740234375
epoch: 22, loss: 1565.0322265625
epoch: 23, loss: 1564.7764892578125
epoch: 24, loss: 1564.499755859375
epoch: 25, loss: 1564.32080078125
epoch: 26, loss: 1564.0953369140625
epoch: 27, loss: 1563.86181640625
epoch: 28, loss: 1563.69189453125
epoch: 29, loss: 1563.58984375
epoch: 30, 

#Failed to interpret file './data/train/label_16_03.tif' as a pickle
#무슨 뜻인지 1시간 정도 고민했는데, image file이 numpy형태로 변환이 안돼서 그런게 아닐까하는 생각을 했다.
#transform이 잘 안되고 있다. tif 파일을 넘파이 배열로 저장시켜야 하는데 그게 안되기 때문에 np.load가 안되고 있는 상황
#transform 은 이상 없었음. 파일을 넘파이로 바꿔주니 잘 넘어감
#넘어는 가는데 input size를 (1,512,512)로 설정해서 train을 하고자 하였지만 들어가는 size가 (4,1,512,512)가 된다. 이유 확인하자
#input format은 이게 맞다. NotImplementedError가 나온건 내가 forward 메소드를 __init__안에다 정의해서 그렇다. 
#그러나 이젠 다른 에러가 나타났다. 맙소사
#뭐였더라, crop 이 안됐었는데 nn.module 내에서 transforms.CenterCrop을 정의했던게 문제였다.
#train은 성공, 
#cuda0 이면 tensor -> numpy 로 전환할 때 >>> x.detach().cpu().numpy() 를 해야 바뀐다. 쿠다 아니면 x.numpy()바로 가능

In [87]:
#Let's test
dict_model = torch.load('%s/%s' % (ckpt_dir,'model_epoch50.pth'))

net.load_state_dict(dict_model['net'])
cnt = 1
# test set도 동일하게 진행
dataset_test = Dataset(data_dir=os.path.join(data_dir,'test'),transform = ToTensor())
#loader_test = DataLoader(dataset_test, batch_size=1 , shuffle=False)
#for batch, data in enumerate(loader_test):
x = dataset_test[0]['input']
y = x.unsqueeze_(0)
print(y.size())

    # forward
#    label = data['label'].to(device)
inputs = y.to(device)
output = net(inputs)

output = fn_tonumpy(output)
#output = 255*output.astype(np.uint8)
io.imsave('prob_map4.tif',np.squeeze(output))
#     loss = fn_loss(fn_classifier(output),label)
#     acc = (fn_classifier(output) == label).float().sum()
#     print("loss is %f \n accuracy is %f " %(loss,acc/(512*512)))

# img.save('%s/%s/result%d.tif' %(data_dir,res_dir,cnt))
# cnt += 1
# plt.imshow(np.squeeze(output))#, interpolation='nearest')
# plt.show()

torch.Size([1, 1, 64, 1024, 1024])


In [26]:
print(loader_test)